In [30]:
# We'll need re later

import re

We can still represent the current state with a dictionary with three parameters, but we'll also need a list of visited locations which we can represent as <code>(x, y)</code> pairs.

* <code>facing</code> $\in \{$ <code>N</code>, <code>S</code>, <code>E</code>, <code>W</code> $\}$
* <code>x</code> $\in Z$
* <code>y</code> $\in Z$


We will also need two functions, each of which takes a value and a state, and returns an updated state:

* <code>turn</code>
* <code>move</code>

<code>turn</code> is the same as before:

In [31]:
def turn(direction, stateIn):
    """
    Should have a docstring here...
    """
    stateOut=stateIn.copy()
    if direction=='L':
        if stateOut['facing']=='N':
            stateOut.update({'facing':'W'})
        elif stateOut['facing']=='W':
            stateOut.update({'facing':'S'})
        elif stateOut['facing']=='S':
            stateOut.update({'facing':'E'})
        elif stateOut['facing']=='E':
            stateOut.update({'facing':'N'})
        else:
            raise ValueError("facing should be one of 'N', 'S', 'E', 'W'")
    elif direction=='R':
        if stateOut['facing']=='N':
            stateOut.update({'facing':'E'})
        elif stateOut['facing']=='E':
            stateOut.update({'facing':'S'})
        elif stateOut['facing']=='S':
            stateOut.update({'facing':'W'})
        elif stateOut['facing']=='W':
            stateOut.update({'facing':'N'})
        else:
            raise ValueError("facing should be one of 'N', 'S', 'E', 'W'")
    else:
        raise ValueError("direction should be 'L' or 'R'")
    return stateOut
            
            
            

but for <code>move</code>, we'll just go one step at a time, and put the multiple steps into the parsing step below.

In [32]:
def move(stateIn):
    """
    And here...
    """
    stateOut=stateIn.copy()
    if stateOut['facing']=='N':
        stateOut.update({'y':stateIn['y']+1})
    elif stateOut['facing']=='S':
        stateOut.update({'y':stateIn['y']-1})
    elif stateOut['facing']=='E':
        stateOut.update({'x':stateIn['x']+1})
    elif stateOut['facing']=='W':
        stateOut.update({'x':stateIn['x']-1})
    else:
        raise ValueError("Invalid state: facing should be one of 'N', 'S', 'E', 'W'")

    return stateOut

Now, rather than go through all the steps, we can use a <code>while</code> loop to stop once we hit the same point again.

Start with a simple example from the problem spec (<code>R8, R4, R4, R8</code>):

In [33]:
inputString='R8, R4, R4, R8'

I'll use the <code>findall</code> function in <code>re</code> to parse the input sequence. This time, want each state to be either a turn or a direction:

In [34]:
re.findall('L|R|\d+', inputString)

['R', '8', 'R', '4', 'R', '4', 'R', '8']

In [35]:
state_dict={'facing':'N', 'x':0, 'y':0}
visited_set=set()

parsedInput_ls=re.findall('L|R|\d+', inputString)

move_i=0 # How many steps left to move

while (state_dict['x'], state_dict['y']) not in visited_set:

    if move_i==0:   # If finished moving, get the next instruction
        
        nextMove_str=parsedInput_ls.pop(0)
        if nextMove_str=='L':
            state_dict=turn('L', state_dict)
        elif nextMove_str=='R':
            state_dict=turn('R', state_dict)
        else:
            move_i=int(nextMove_str)   # Will raise an error if not an int
            
    else:
        visited_set.add((state_dict['x'], state_dict['y']))
        state_dict=move(state_dict)
        move_i -= 1

state_dict

{'facing': 'N', 'x': 4, 'y': 0}

In [36]:
visited_set

{(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, -4),
 (4, -3),
 (4, -2),
 (4, -1),
 (4, 0),
 (5, -4),
 (5, 0),
 (6, -4),
 (6, 0),
 (7, -4),
 (7, 0),
 (8, -4),
 (8, -3),
 (8, -2),
 (8, -1),
 (8, 0)}

Finally, need to calculate the total number of blocks away, by adding the absolute values of the x and y coords:

In [37]:
abs(state_dict['x'])+abs(state_dict['y'])

4

Good. Now do the same thing with the input from the site:

In [38]:
inputString='R1, L3, R5, R5, R5, L4, R5, R1, R2, L1, L1, R5, R1, L3, L5, L2, R4, L1, R4, R5, L3, R5, L1, R3, L5, R1, L2, R1, L5, L1, R1, R4, R1, L1, L3, R3, R5, L3, R4, L4, R5, L5, L1, L2, R4, R3, R3, L185, R3, R4, L5, L4, R48, R1, R2, L1, R1, L4, L4, R77, R5, L2, R192, R2, R5, L4, L5, L3, R2, L4, R1, L5, R5, R4, R1, R2, L3, R4, R4, L2, L4, L3, R5, R4, L2, L1, L3, R1, R5, R5, R2, L5, L2, L3, L4, R2, R1, L4, L1, R1, R5, R3, R3, R4, L1, L4, R1, L2, R3, L3, L2, L1, L2, L2, L1, L2, R3, R1, L4, R1, L1, L4, R1, L2, L5, R3, L5, L2, L2, L3, R1, L4, R1, R1, R2, L1, L4, L4, R2, R2, R2, R2, R5, R1, L1, L4, L5, R2, R4, L3, L5, R2, R3, L4, L1, R2, R3, R5, L2, L3, R3, R1, R3'

In [39]:
state_dict={'facing':'N', 'x':0, 'y':0}
visited_set=set()

parsedInput_ls=re.findall('L|R|\d+', inputString)

move_i=0 # How many steps left to move

while (state_dict['x'], state_dict['y']) not in visited_set:

    if move_i==0:   # If finished moving, get the next instruction
        
        nextMove_str=parsedInput_ls.pop(0) # Raises an error if out of instructions
        if nextMove_str=='L':
            state_dict=turn('L', state_dict)
        elif nextMove_str=='R':
            state_dict=turn('R', state_dict)
        else:
            move_i=int(nextMove_str)   # Will raise an error if not an int
            
    else:
        visited_set.add((state_dict['x'], state_dict['y']))
        state_dict=move(state_dict)
        move_i -= 1

abs(state_dict['x'])+abs(state_dict['y'])

158